# 1.0 Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## 1.1 Funcões

In [2]:
# Função para calcular métricas
def calcular_metricas(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "MSE": mean_squared_error(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "MAE": mean_absolute_error(y_true, y_pred),
        "MAPE":mean_absolute_percentage_error(y_true, y_pred)
    }

# Função para realizar Grid Search
def grid_search(model, param_grid, X_train, y_train):
    grid_search = GridSearchCV(model, param_grid)
    grid_search.fit(X_train, y_train)
    return grid_search.best_params_

# 2.0 Leitura dos Dados

In [3]:
x_train = pd.read_csv( '../datasets/regression/X_training.csv' )
y_train = pd.read_csv( '../datasets/regression/y_training.csv' )

x_val = pd.read_csv( '../datasets/regression/X_validation.csv' )
y_val = pd.read_csv( '../datasets/regression/y_validation.csv' )

x_test = pd.read_csv( '../datasets/regression/X_test.csv' )
y_test = pd.read_csv( '../datasets/regression/y_test.csv' )

## 2.1 Verificando melhores parametros para treinamento do modelo.

In [4]:
# Definindo os modelos
rf_model = RandomForestRegressor()
lrr_model = Ridge()
lrl_model = Lasso()
lren_model = ElasticNet()
dt_model = DecisionTreeRegressor()

# Definindo as grades de parâmetros
rf_param_grid = {'n_estimators': [10, 50, 100, 200], 'max_depth': [None, 10, 20, 30]}
lrr_param_grid = {'alpha': [1.0, 2.0, 5.0, 10.0], 'max_iter': [None, 10, 20, 30]}
lrl_param_grid = {'alpha': [1.0, 2.0, 5.0, 10.0], 'max_iter': [None, 10, 20, 30]}
lren_param_grid = {'alpha': [1.0, 2.0, 5.0, 10.0], 'l1_ratio': [0.0, 0.5, 1.0], 'max_iter': [None, 10, 20, 30]}
dt_param_grid = {'max_depth': [None, 10, 20, 30, 40]}

In [5]:
# Random Forest
rf_best_params = grid_search(rf_model, rf_param_grid, x_train, y_train)
print("Melhores parâmetros para Random Forest:", rf_best_params)

# Linear Regression Ridge
lrr_best_params = grid_search(lrr_model, lrr_param_grid, x_train, y_train)
print("Melhores parâmetros para Linear Regression Ridge:", lrr_best_params)

# Linear Regression Lasso
lrl_best_params = grid_search(lrl_model, lrl_param_grid, x_train, y_train)
print("Melhores parâmetros para Linear Regression Lasso:", lrl_best_params)

# Linear Regression Elastic Net
lren_best_params = grid_search(lren_model, lren_param_grid, x_train, y_train)
print("Melhores parâmetros para Linear Regression Elastic Net:", lren_best_params)

# Decision Tree
dt_best_params = grid_search(dt_model, dt_param_grid, x_train, y_train)
print("Melhores parâmetros para Decision Tree:", dt_best_params)

Melhores parâmetros para Random Forest: {'max_depth': 30, 'n_estimators': 200}
Melhores parâmetros para Linear Regression Ridge: {'alpha': 5.0, 'max_iter': None}
Melhores parâmetros para Linear Regression Lasso: {'alpha': 1.0, 'max_iter': 10}
Melhores parâmetros para Linear Regression Elastic Net: {'alpha': 1.0, 'l1_ratio': 0.0, 'max_iter': 20}
Melhores parâmetros para Decision Tree: {'max_depth': 10}


# 3.0 Treinando os Modelos

In [6]:
# Definindo os modelos
modelos = {
    "Decision Tree": DecisionTreeRegressor(max_depth=10),
    "Random Forest": RandomForestRegressor(max_depth=None, n_estimators=200),
    "Logistic Regression": LinearRegression(),
    "Logistic Regression Ridge": Ridge(alpha=5.0, max_iter=None),
    "Logistic Regression Lasso": Lasso(alpha=1.0, max_iter=10),
    "Logistic Regression Elastic Net": ElasticNet(alpha=1.0, l1_ratio=0.0, max_iter=20)
}
# Treinando os modelos e armazenando as métricas de treinamento e validação
resultados_treinamento = []
resultados_validacao = []

for nome, modelo in modelos.items():
    # Treinamento
    modelo.fit(x_train, y_train)
    y_pred_treino = modelo.predict(x_train)
    metricas_treino = calcular_metricas(y_train, y_pred_treino)
    resultados_treinamento.append({"Modelo": nome, **metricas_treino})
    print(f"Modelo {nome} treinado.")

    # Validação
    y_pred_val = modelo.predict(x_val)
    metricas_val = calcular_metricas(y_val, y_pred_val)
    resultados_validacao.append({"Modelo": nome, **metricas_val})

Modelo Decision Tree treinado.
Modelo Random Forest treinado.
Modelo Logistic Regression treinado.
Modelo Logistic Regression Ridge treinado.
Modelo Logistic Regression Lasso treinado.
Modelo Logistic Regression Elastic Net treinado.


# 4.0 Performance

In [7]:
# Criando tabelas de resultados
tabela_treinamento = pd.DataFrame(resultados_treinamento)
tabela_validacao = pd.DataFrame(resultados_validacao)

# Exibindo os resultados
print("Resultados de Treinamento:")
print(tabela_treinamento)
print("\nResultados de Validação:")
print(tabela_validacao)

Resultados de Treinamento:
                            Modelo        R2         MSE       RMSE  \
0                    Decision Tree  0.384624  294.157341  17.151016   
1                    Random Forest  0.904832   45.491423   6.744733   
2              Logistic Regression  0.046058  455.996112  21.354065   
3        Logistic Regression Ridge  0.046045  456.002667  21.354219   
4        Logistic Regression Lasso  0.007401  474.474834  21.782443   
5  Logistic Regression Elastic Net  0.010715  472.890766  21.746052   

         MAE      MAPE  
0  12.925051  4.871412  
1   4.823036  2.621872  
2  16.998249  8.653186  
3  16.998596  8.654275  
4  17.305484  8.736697  
5  17.277098  8.719880  

Resultados de Validação:
                            Modelo        R2         MSE       RMSE  \
0                    Decision Tree -0.010764  482.651378  21.969328   
1                    Random Forest  0.336046  317.045637  17.805775   
2              Logistic Regression  0.039925  458.447042  21.

# Dados de Teste

In [8]:
# Avaliação no conjunto de teste
resultados_teste = []

for nome, modelo in modelos.items():
    y_pred_teste = modelo.predict(x_test)
    metricas_teste = calcular_metricas(y_test, y_pred_teste)
    resultados_teste.append({"Modelo": nome, **metricas_teste})

tabela_teste = pd.DataFrame(resultados_teste)
print("Resultados no Conjunto de Teste:")
print(tabela_teste)

Resultados no Conjunto de Teste:
                            Modelo        R2         MSE       RMSE  \
0                    Decision Tree  0.036638  469.062024  21.657840   
1                    Random Forest  0.353931  314.571659  17.736168   
2              Logistic Regression  0.052317  461.427719  21.480869   
3        Logistic Regression Ridge  0.052265  461.452970  21.481456   
4        Logistic Regression Lasso  0.007646  483.177970  21.981310   
5  Logistic Regression Elastic Net  0.010636  481.722175  21.948170   

         MAE      MAPE  
0  16.822880  6.929010  
1  13.020025  6.585227  
2  17.129965  8.521859  
3  17.128819  8.526457  
4  17.472989  8.753477  
5  17.447495  8.731736  


# 5.0 Polinomial Regression

## 5.1 Testando o melhor grau para o treinamento do modelo

In [9]:
#intervalo de graus para testar.
pparam_grid = {'polynomialfeatures__degree': np.arange(1, 5)}

#pipeline com a inclusão da transformação polinomial e a regressão linear.
pipeline = Pipeline([
    ('polynomialfeatures', PolynomialFeatures()),
    ('linearregression', LinearRegression())
])

#testar todos os graus no intervalo definido para avaliar o desempenho.
pgrid_search = GridSearchCV(pipeline, pparam_grid, cv=3)

#treinamento
pgrid_search.fit(x_train, y_train)

# verificar desempenho
print("Melhor grau:", pgrid_search.best_params_)
print("Melhor score:", pgrid_search.best_score_)

Melhor grau: {'polynomialfeatures__degree': 2}
Melhor score: 0.06999317918067653


## 5.2 Transformação Polinomial dos Dados

In [10]:
poly = PolynomialFeatures(degree=2)

x_train_poly = poly.fit_transform(x_train)
x_val_poly = poly.transform(x_val)
x_test_poly = poly.transform(x_test)

## 5.3 Verificando melhores parametros para treinamento do modelo.

In [11]:
# Definindo os modelos
plrr_model = Ridge()
plrl_model = Lasso()
plren_model = ElasticNet()

# Definindo as grades de parâmetros
plrr_param_grid = {'alpha': [1.0, 2.0, 5.0, 10.0], 'max_iter': [None, 10, 20, 30]}
plrl_param_grid = {'alpha': [1.0, 2.0, 5.0, 10.0], 'max_iter': [None, 10, 20, 30]}
plren_param_grid = {'alpha': [1.0, 2.0, 5.0, 10.0], 'l1_ratio': [0.0, 0.5, 1.0], 'max_iter': [None, 10, 20, 30]}

In [12]:
# Polinomial Regression Ridge
plrr_best_params = grid_search(plrr_model, plrr_param_grid, x_train_poly, y_train)
print("Melhores parâmetros para Polinomial Regression Ridge:", plrr_best_params)

# Polinomial Regression Lasso
plrl_best_params = grid_search(plrl_model, plrl_param_grid, x_train_poly, y_train)
print("Melhores parâmetros para Polinomial Regression Lasso:", plrl_best_params)

# Polinomial Regression Elastic Net
plren_best_params = grid_search(plren_model, plren_param_grid, x_train_poly, y_train)
print("Melhores parâmetros para Polinomial Regression Elastic Net:", plren_best_params)

Melhores parâmetros para Polinomial Regression Ridge: {'alpha': 2.0, 'max_iter': None}
Melhores parâmetros para Polinomial Regression Lasso: {'alpha': 1.0, 'max_iter': 20}
Melhores parâmetros para Polinomial Regression Elastic Net: {'alpha': 1.0, 'l1_ratio': 0.0, 'max_iter': 30}


# 5.4 Treinando os Modelos Polinomiais

In [13]:
# Definindo os modelos
modelos = {
    "Polinomial Regression": LinearRegression(),
    "Polinomial Regression Ridge": Ridge(alpha=2.0, max_iter=None),
    "Polinomial Regression Lasso": Lasso(alpha=1.0, max_iter=20),
    "Polinomial Regression Elastic Net": ElasticNet(alpha=1.0, l1_ratio=0.0, max_iter=30)
}
# Treinando os modelos e armazenando as métricas de treinamento e validação
resultados_treinamento = []
resultados_validacao = []

for nome, modelo in modelos.items():
    # Treinamento
    modelo.fit(x_train_poly, y_train)
    y_pred_treino = modelo.predict(x_train_poly)
    metricas_treino = calcular_metricas(y_train, y_pred_treino)
    resultados_treinamento.append({"Modelo": nome, **metricas_treino})
    print(f"Modelo {nome} treinado.")

    # Validação
    y_pred_val = modelo.predict(x_val_poly)
    metricas_val = calcular_metricas(y_val, y_pred_val)
    resultados_validacao.append({"Modelo": nome, **metricas_val})

Modelo Polinomial Regression treinado.
Modelo Polinomial Regression Ridge treinado.
Modelo Polinomial Regression Lasso treinado.
Modelo Polinomial Regression Elastic Net treinado.


## 5.5 Performance

In [14]:
# Criando tabelas de resultados
tabela_treinamento = pd.DataFrame(resultados_treinamento)
tabela_validacao = pd.DataFrame(resultados_validacao)

# Exibindo os resultados
print("Resultados de Treinamento:")
print(tabela_treinamento)
print("\nResultados de Validação:")
print(tabela_validacao)

Resultados de Treinamento:
                              Modelo        R2         MSE       RMSE  \
0              Polinomial Regression  0.094195  432.986210  20.808321   
1        Polinomial Regression Ridge  0.092544  433.775338  20.827274   
2        Polinomial Regression Lasso  0.009150  473.638776  21.763244   
3  Polinomial Regression Elastic Net  0.026167  465.504391  21.575551   

         MAE      MAPE  
0  16.458032  8.350540  
1  16.479345  8.382553  
2  17.285450  8.699701  
3  17.120834  8.637742  

Resultados de Validação:
                              Modelo        R2         MSE       RMSE  \
0              Polinomial Regression  0.066477  445.768223  21.113224   
1        Polinomial Regression Ridge  0.067695  445.186573  21.099445   
2        Polinomial Regression Lasso  0.009631  472.912694  21.746556   
3  Polinomial Regression Elastic Net  0.023532  466.274996  21.593402   

         MAE      MAPE  
0  16.749939  8.547931  
1  16.739734  8.575067  
2  17.238379  8

## Dados de Teste

In [15]:
# Avaliação no conjunto de teste
resultados_teste = []

for nome, modelo in modelos.items():
    y_pred_teste = modelo.predict(x_test_poly)
    metricas_teste = calcular_metricas(y_test, y_pred_teste)
    resultados_teste.append({"Modelo": nome, **metricas_teste})

tabela_teste = pd.DataFrame(resultados_teste)
print("Resultados no Conjunto de Teste:")
print(tabela_teste)

Resultados no Conjunto de Teste:
                              Modelo        R2         MSE       RMSE  \
0              Polinomial Regression  0.090079  443.041256  21.048545   
1        Polinomial Regression Ridge  0.088422  443.848061  21.067702   
2        Polinomial Regression Lasso  0.008372  482.824405  21.973266   
3  Polinomial Regression Elastic Net  0.021274  476.542822  21.829861   

         MAE      MAPE  
0  16.720535  8.242464  
1  16.735629  8.304551  
2  17.456854  8.756072  
3  17.313045  8.704860  
